In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import sys
import os
import numpy as np
import pandas as pd
import scipy.sparse
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Circle
import seaborn as sns; 
from IPython.display import HTML

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import r2_score

In [ ]:
#config parser
import configparser

sys.path.insert(0, '../Common/')
from AISDataManager import AISDataManager
import Constants as c
import HMUtils as hMUtil
import TimeUtils as timeUtils
import GeoCompute as gC

#MyConfig.INI stores all the run time constants
config = configparser.ConfigParser()
config.read('../MyConfig.INI')

from joblib import Parallel, delayed
import multiprocessing
aISDM = AISDataManager()
numCores = multiprocessing.cpu_count()

In [ ]:
lonMin = (float)(config['TP_SEC_ORDER']['LON_MIN'])
lonMax = (float)(config['TP_SEC_ORDER']['LON_MAX'])

latMin = (float)(config['TP_SEC_ORDER']['LAT_MIN'])
latMax = (float)(config['TP_SEC_ORDER']['LAT_MAX'])

print(lonMin,latMin)
print(lonMax,latMax)

increStep = (float)(config['TP_SEC_ORDER']['INCR_STEP'])
incrRes = (int)(config['TP_SEC_ORDER']['INCR_RES'])

fileDir = config['TP_SEC_ORDER']['SOURCE_DIR']
dirToStore = config['TP_SEC_ORDER']['DEST_DIR']

print("SOURCE_DIR = %s"%fileDir)
print("DEST_DIR = %s"%dirToStore)

In [ ]:
heatMapGrid = hMUtil.generate_grid(lonMin, lonMax, latMin, latMax, increStep, incrRes)

In [ ]:
boundaryArray = heatMapGrid[2]
horizontalAxis = heatMapGrid[0]
verticalAxis = heatMapGrid[1]
totalStates = horizontalAxis.shape[0] * verticalAxis.shape[0]

In [ ]:
opFile = dirToStore + 'TPM.npz'
# opFile = dirToStore + 'TPM_.npz'
tpMatrix = scipy.sparse.load_npz(opFile)

# opFile = dirToStore + 'TPM.npy'
# tpMatrix = np.load(opFile)

In [ ]:
print(tpMatrix.sum())

In [ ]:
def get_vessel_init_status(lonStart,latStart):
    vesselState = np.zeros((horizontalAxis.shape[0]*verticalAxis.shape[0]))
    for boundary in boundaryArray :
        if((lonStart >= boundary[0]) \
            and (lonStart < boundary[1]) \
           
            and (latStart >= boundary[2]) \
            and (latStart < boundary[3]) \
          ):
            vesselState[boundary[4]] = 1.0
            break
    return vesselState

In [ ]:
#takes inital state vector
#and number of steps 
#uses transition probabilities to generate the path
def generate_path(vesselStatus):
    return (vesselStatus.T @ tpMatrix).flatten()

def generate_vessel_prev_cur_status(prevState,currState):
    vesselState = np.zeros((totalStates,totalStates))
    for i in range(prevState.shape[0]):
        vesselState[i,:] = (prevState[i]*currState)
    return vesselState.flatten()

In [ ]:
oneVessel = "../Data/M120_50_M119_00_33_90_34_44/MMSI_17_1004_G15/2732.csv"
oneVesselLastDataOne,_ = aISDM.load_data_from_csv(oneVessel)

In [ ]:
print(oneVesselLastDataOne)

In [ ]:
fix,ax = plt.subplots()
oneVesselLastDataOneLon = oneVesselLastDataOne["LON"]
oneVesselLastDataOneLat = oneVesselLastDataOne["LAT"]
ax.set_xlim(lonMin, lonMax)
ax.set_ylim(latMin, latMax)
ax.plot(oneVesselLastDataOneLon,oneVesselLastDataOneLat)

In [ ]:
rowSum = np.sum(tpMatrix, axis=1)
rowSumZeroIdx = np.argwhere(rowSum == 0)
rowSumZeroIdx = rowSumZeroIdx.flatten()

In [ ]:
vesselPrevState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[0],oneVesselLastDataOneLat.iloc[0])
vesselCurrState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[1],oneVesselLastDataOneLat.iloc[1])
vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)
onePredict = []
for i in range(6):
    vesselProbState = generate_path(vesselState)
    print(i)
    print(np.sum(vesselProbState))
    onePredict.append(vesselProbState.copy())
    vesselPrevState = vesselCurrState.copy()
    vesselCurrState = vesselProbState.copy()
    vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)    
    #make probabilities zero
    vesselState[rowSumZeroIdx] = 0
    vesselState = vesselState/np.sum(vesselState)
    print(np.sum(vesselState))

In [ ]:
vesselPrevState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[0],oneVesselLastDataOneLat.iloc[0])
vesselCurrState = get_vessel_init_status(oneVesselLastDataOneLon.iloc[1],oneVesselLastDataOneLat.iloc[1])
vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)
useAspect = 0
aspectRatio = 16.0/9.0
figWidth = 10
figHeight = 8

fig = plt.figure(figsize=(figWidth,figHeight))
ax1 = fig.add_axes([0.1, 0.15, 0.49, 0.30])
ax2 = fig.add_axes([0.1, 0.55, 0.49, 0.30])
ax4 = fig.add_axes([0.8, 0.55, 0.01, 0.30])

if useAspect == 1:
    figHeight = figWidth/aspectRatio

def animate(i):
    global vesselPrevState, vesselCurrState, vesselState
    vesselCurrPos = get_vessel_init_status(oneVesselLastDataOneLon.iloc[i+2],oneVesselLastDataOneLat.iloc[i+2])
    vesselCurrPos = np.reshape(vesselCurrPos,(verticalAxis.shape[0],horizontalAxis.shape[0]))
    
    vesselCurrPos = np.flip(vesselCurrPos,axis=0)

    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Lattitude')
    titleStr = 'Vessel Position %d'%((i))
    ax1.set_title(titleStr)
    ax1.set_xticks([lonMin,lonMax])
    ax1.set_yticks([latMin,latMax])

    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('Lattitude')
    ax2.set_title('Transition Probability Projection')
    ax2.set_xticks([lonMin,lonMax])
    ax2.set_yticks([latMin,latMax])

    ax1.imshow(vesselCurrPos,cmap=plt.get_cmap('hot'),extent=[lonMin,lonMax,latMin,latMax])
    #sequence of vessel states
#     vesselProbState = generate_path(vesselState)
#     #copy the file
#     vesselPrevState = vesselCurrState.copy()
#     vesselCurrState = vesselProbState.copy()
#     vesselState = generate_vessel_prev_cur_status(vesselPrevState,vesselCurrState)
    
    vesselProbState = onePredict[i].copy()
    vesselProbState = np.reshape(vesselProbState,(verticalAxis.shape[0],horizontalAxis.shape[0]))
    vesselProbState = np.flip(vesselProbState,axis=0)
    iMShowHandler = ax2.imshow(vesselProbState,cmap=plt.get_cmap('YlGnBu'),extent=[lonMin,lonMax,latMin,latMax], vmax = 0.1)
    
    fig.colorbar(iMShowHandler,ax4)
    
anim = animation.FuncAnimation(fig, animate,frames=6, interval=1000)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=1, metadata=dict(artist='Me'), bitrate=1800)

In [ ]:
# anim.save('TPSeqOneSecOrder.mp4', writer=writer)
# anim.save('TPSeqTwoSecOrder.mp4', writer=writer)
# anim.save('FishingVessel.mp4', writer=writer)
anim.save('NewRegion3.mp4', writer=writer)